### Previously on
* Supervised: regression and classification
* Unsupervised: clustering, reduce dimensionality
* Clustering is a key way to explore unknown data, and it's a very commonly used machine learning technique

### US senator data

In [ ]:
# Determine which senators are more or less in the mainstream of their party

import pandas as pd
votes=pd.read_csv('114_congress.csv')

In [ ]:
# Exploring
print(votes.party.value_counts())
votes.mean()

# If the mean for a column is less than .5, more Senators voted against the bill, and vice versa if it's over .5. Print the results.


In [ ]:
# Euclidean distance between senators using only vote columns
from sklearn.metrics.pairwise import euclidean_distances

print(euclidean_distances(votes.iloc[0,3:].values.reshape(1, -1), votes.iloc[1,3:].values.reshape(1, -1)))

distance=euclidean_distances(votes.iloc[0,3:], votes.iloc[2,3:])

### Notes
* Because we aren't predicting anything, there's no risk of overfitting, so we'll train our model on the whole dataset.
* The crosstab() method takes in two vectors or Pandas Series and computes how many times each unique value in the second vector occurs for each unique value in the first vector.


### K-Mean clustering

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans

kmeans_model = KMeans(n_clusters=2, random_state=1)
senator_distances=kmeans_model.fit_transform(votes.iloc[:,3:])

In [ ]:
# Use crosstab: occurence of same values for two vectors
labels=kmeans_model.labels_
pd.crosstab(labels,votes['party'])

In [ ]:
democratic_outliers=votes[(labels==1) & (votes["party"] == "D")]
print(democratic_outliers)

In [ ]:
# Visualize the clusters
plt.scatter(senator_distances[:,0],senator_distances[:,1],c=labels,lw=0)
plt.show()

In [ ]:
import numpy as np
extremism=(senator_distances**3).sum(axis=1)
votes['extremism']=extremism

votes.sort_values('extremism',inplace=True,ascending=False)
print(votes.head(10))

### NBA K-Means clustering >> recreate the K-Means clustering process
3 steps:
(1) Assign each point to a clustter (evaluate the distance to randomly assigned centers)
(2) Re-evaluate the points in each clustter to find new centers
(3) Iterate the above processes

In [ ]:
import pandas as pd
import numpy as np
nba = pd.read_csv("nba_2013.csv")
nba.head(3)


In [ ]:
# Filter out point_guards
point_guards=nba[nba['pos']=='PG'].copy()

# Create a new feature points per game
point_guards['ppg'] = point_guards['pts'] / point_guards['g']

# Sanity check, make sure ppg = pts/g
point_guards[['pts', 'g', 'ppg']].head(5)

# Assist-turnover ratio
point_guards = point_guards[point_guards['tov'] != 0]
point_guards['atr']=point_guards['ast']/point_guards['tov']

In [ ]:
plt.scatter(point_guards['ppg'], point_guards['atr'], c='y')
plt.title("Point Guards")
plt.xlabel('Points Per Game', fontsize=13)
plt.ylabel('Assist Turnover Ratio', fontsize=13)
plt.show()

In [ ]:
num_clusters = 5
# Use numpy's random function to generate a list, length: num_clusters, of indices
random_initial_points = np.random.choice(point_guards.index, size=num_clusters)
# Use the random indices to create the centroids
centroids = point_guards.loc[random_initial_points]

In [ ]:
# Visualize the five randomly selected centroids
plt.scatter(point_guards['ppg'], point_guards['atr'], c='yellow')
plt.scatter(centroids['ppg'], centroids['atr'], c='red')
plt.title("Centroids")
plt.xlabel('Points Per Game', fontsize=13)
plt.ylabel('Assist Turnover Ratio', fontsize=13)
plt.show()

In [ ]:
def centroids_to_dict(centroids):
    dictionary = dict()
    # iterating counter we use to generate a cluster_id
    counter = 0

    # iterate a pandas data frame row-wise using .iterrows()
    for index, row in centroids.iterrows():
        coordinates = [row['ppg'], row['atr']]
        dictionary[counter] = coordinates
        counter += 1

    return dictionary

centroids_dict = centroids_to_dict(centroids)

In [ ]:
#Euclidean distance
import math

def calculate_distance(centroid, player_values):
    root_distance = 0
    
    for x in range(0, len(centroid)):
        difference = centroid[x] - player_values[x]
        squared_difference = difference**2
        root_distance += squared_difference

    euclid_distance = math.sqrt(root_distance)
    return euclid_distance

q = [5, 2]
p = [3,1]

# Sqrt(5) = ~2.24
print(calculate_distance(q, p))

In [ ]:
# Add the function, `assign_to_cluster`
# This creates the column, `cluster`, by applying assign_to_cluster row-by-row

def assign_to_cluster(row):
    
    player_values=[row.ppg,row.atr]
    dist=[]
    for idx, centroid in centroids_dict.items():
        euclidean_distance=calculate_distance(centroid,player_values)
        dist.append(euclidean_distance)
    cluster_id=np.argmin(dist)
    return cluster_id

point_guards['cluster'] = point_guards.apply(lambda row: assign_to_cluster(row), axis=1)



In [ ]:
# Visualizing clusters
def visualize_clusters(df, num_clusters):
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

    for n in range(num_clusters):
        clustered_df = df[df['cluster'] == n]
        plt.scatter(clustered_df['ppg'], clustered_df['atr'], c=colors[n-1])
        plt.xlabel('Points Per Game', fontsize=13)
        plt.ylabel('Assist Turnover Ratio', fontsize=13)
    plt.show()

visualize_clusters(point_guards, 5)

In [ ]:
def recalculate_centroids(df):
    new_centroids_dict = dict()
    # 0..1...2...3...4
    for cluster_id in range(0, num_clusters):
        # Finish the logic
        partial=df[df['cluster']==cluster_id]
        new_centroids_dict[cluster_id]=[partial['ppg'].mean(),partial['atr'].mean()]
        
        return new_centroids_dict

centroids_dict = recalculate_centroids(point_guards)

In [ ]:
# Rerun assign to cluster based on new centroids:
point_guards['cluster'] = point_guards.apply(lambda row: assign_to_cluster(row), axis=1)
visualize_clusters(point_guards, num_clusters)

### Notes
To counteract these problems, the sklearn implementation of K-Means does some intelligent things like re-running the entire clustering process lots of times with random initial centroids so the final results are a little less biased on one passthrough's initial centroids.

* For many machine learning algorithms it's important to scale, or normalize, the data before using it.

### Gradient Descent
* PGA data, the negative correlation between accuracy and distance
* Gradient descent is a general method that can be used to estimate coefficents of nearly any model, including linear models. At it's core, gradient descent minimizes the residuals in the estimated model by updating each coefficent based on it's gradient.
* What is the cost function? For Linear regression $J(\theta_{0},\theta_{1})=\frac{1}{2m}\sum\limits_{i=1}^m(h_{\theta}(x_{i})-y_{i})^{2}$
* The minimum of the cost function is the point where the model has the lowest error, hence the point where our parameters are optimized.
* Gradient descent relies on finding the direction of the largest gradient where a gradient is the "slope" of a multivariable function. To find this gradient we can take the partial derivative in terms of each parameter in the cost function. 


In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

# We can add a dimension to an array by using np.newaxis
print("Shape of the series:", pga['distance'].shape)
print("Shape with newaxis:", pga['distance'][:, np.newaxis].shape)

# The X variable in LinearRegression.fit() must have 2 dimensions
model=LinearRegression()
model.fit(pga[['distance']],pga['accuracy'])

theta1=model.coef_

In [ ]:
# The cost function of a single variable linear model
def cost(theta0, theta1, x, y):
    # Initialize cost
    J = 0
    # The number of observations
    m = len(x)
    # Loop through each observation
    for i in range(m):
        # Compute the hypothesis 
        h = theta1 * x[i] + theta0
        # Add to cost
        J += (h - y[i])**2
    # Average and normalize cost
    J /= (2*m)
    return J

# The cost for theta0=0 and theta1=1
print(cost(0, 1, pga['distance'], pga['accuracy']))

theta0 = 100
theta1s = np.linspace(-3,2,100)
costs=cost(theta0,theta1s,pga['distance'], pga['accuracy'])

plt.plot(theta1s,costs)
plt.show()

In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

# Example of a Surface Plot using Matplotlib
# Create x an y variables
x = np.linspace(-10,10,100)
y = np.linspace(-10,10,100)

# We must create variables to represent each possible pair of points in x and y
# ie. (-10, 10), (-10, -9.8), ... (0, 0), ... ,(10, 9.8), (10,9.8)
# x and y need to be transformed to 100x100 matrices to represent these coordinates
# np.meshgrid will build a coordinate matrices of x and y
X, Y = np.meshgrid(x,y)
print(X[:5,:5],"\n",Y[:5,:5])

# Compute a 3D parabola 
Z = X**2 + Y**2 

# Open a figure to place the plot on
fig = plt.figure()
# Initialize 3D plot
ax = fig.gca(projection='3d')
# Plot the surface
ax.plot_surface(X=X,Y=Y,Z=Z)

plt.show()

# Use these for your exercise
theta0s = np.linspace(-2,2,100)
theta1s = np.linspace(-2,2, 100)
COST = np.empty(shape=(100,100))

# Meshgrid for paramaters 
T0S, T1S = np.meshgrid(theta0s, theta1s)

for i in range(100):
    for j in range(100):
        COST[i,j]=cost(T0S[0,i],T1S[j,0],pga['distance'], pga['accuracy'])
        
# 3d Plot
fig2 = plt.figure()
ax = fig2.gca(projection='3d')
ax.plot_surface(X=T0S,Y=T1S,Z=COST)
plt.show()



In [ ]:
def partial_cost_theta1(theta0, theta1, x, y):
    # Hypothesis
    h = theta0 + theta1*x
    # Hypothesis minus observed times x
    diff = (h - y) * x
    # Average to compute partial derivative
    partial = diff.sum() / (x.shape[0])
    return partial

partial1 = partial_cost_theta1(0, 5, pga['distance'], pga['accuracy'])
print("partial1 =", partial1)


def partial_cost_theta0(theta0, theta1, x, y):
    # Hypothesis
    h = theta0 + theta1*x
    # Hypothesis minus observed times x
    diff = h - y
    # Average to compute partial derivative
    partial0 = diff.sum() / (x.shape[0])
    return partial0

partial0 = partial_cost_theta0(1, 1, pga['distance'], pga['accuracy'])
print("partial0 =", partial0)

#### Notes:
* We measure convergence by finding the difference between the previous iterations cost versus the current cost.
* Formula $\theta_{1}:=\theta_{1}-\alpha \cdot \frac{\partial{J}(\theta_{0},\theta_{1})}{\partial \theta_{1}}$ ; $\theta_{0}:=\theta_{0}-\alpha \cdot \frac{\partial{J}(\theta_{0},\theta_{1})}{\partial \theta_{0}}$
* $\alpha$ is the learning rate: This value is set by the user and controls how fast the algorithm will converge by changing the parameters by some percentage of the slope.

In [ ]:
# x is our feature vector -- distance
# y is our target variable -- accuracy
# alpha is the learning rate
# theta0 is the initial theta0 
# theta1 is the initial theta1
def gradient_descent(x, y, alpha=0.1, theta0=0, theta1=0):
    max_epochs = 1000 # Maximum number of iterations
    counter = 0      # Initialize a counter
    c = cost(theta0, theta1, pga['distance'], pga['accuracy'])  ## Initial cost
    costs = [c]     # Lets store each update
    # Set a convergence threshold to find where the cost function in minimized
    # When the difference between the previous cost and current cost 
    #        is less than this value we will say the parameters converged
    convergence_thres = 0.000001  
    cprev = c + 10   
    theta0s = [theta0]
    theta1s = [theta1]

    # When the costs converge or we hit a large number of iterations will we stop updating
    while (np.abs(cprev - c) > convergence_thres) and (counter < max_epochs):
        cprev = c
        # Alpha times the partial derivative is our updated
        update0 = alpha * partial_cost_theta0(theta0, theta1, x, y)
        update1 = alpha * partial_cost_theta1(theta0, theta1, x, y)

        # Update theta0 and theta1 at the same time
        # We want to compute the slopes at the same set of hypothesised parameters
        #             so we update after finding the partial derivatives
        theta0 -= update0
        theta1 -= update1
        
        # Store thetas
        theta0s.append(theta0)
        theta1s.append(theta1)
        
        # Compute the new cost
        c = cost(theta0, theta1, pga['distance'], pga['accuracy'])

        # Store updates
        costs.append(c)
        counter += 1   # Count

    return {'theta0': theta0, 'theta1': theta1, "costs": costs}

print("Theta1 =", gradient_descent(pga['distance'], pga['accuracy'])['theta1'])

descend = gradient_descent(pga['distance'], pga['accuracy'], alpha=.01)

plt.scatter(range(len(descend["costs"])), descend["costs"])